In [53]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np
import json
from time import sleep, perf_counter as pf
import itertools as it

In [54]:
events = pd.read_csv('datasets/specialEvents.csv')
display(events)

,eventKey,name,isOffWork,type,outcome,startDate,endDate,startTime,endTime,city
0,0,Hockey,0,Sport,1.0,2014-10-08,2014-10-08,22:00:00,NaN,Vancouver
1,1,Hockey,1,Sport,1.0,2014-10-11,2014-10-11,22:00:00,NaN,Vancouver
2,2,Hockey,0,Sport,1.0,2014-10-17,2014-10-17,21:30:00,NaN,Vancouver
3,3,Hockey,1,Sport,2.0,2014-10-18,2014-10-18,22:00:00,NaN,Vancouver
4,4,Hockey,0,Sport,2.0,2014-10-21,2014-10-21,20:30:00,NaN,Vancouver
5,5,Hockey,0,Sport,1.0,2014-10-23,2014-10-23,20:00:00,NaN,Vancouver
6,6,Hockey,0,Sport,2.0,2014-10-24,2014-10-24,21:00:00,NaN,Vancouver
7,7,Hockey,1,Sport,1.0,2014-10-26,2014-10-26,21:30:00,NaN,Vancouver
8,8,Hockey,1,Sport,NaN,2014-10-26,2014-10-26,22:00:00,NaN,Vancouver
9,9,Hockey,0,Sport,1.0,2014-10-28,2014-10-28,22:00:00,NaN,Vancouver


In [55]:
display(events.loc[events['startDate'] != events['endDate']])

,eventKey,name,isOffWork,type,outcome,startDate,endDate,startTime,endTime,city
161,161,MeadowGrass Music Festival,0,Festival,NaN,2015-05-22,2015-05-24,NaN,NaN,Denver
177,177,Italian Heritage Month,0,Festival,NaN,2015-06-01,2015-06-30,NaN,NaN,Vancouver
178,178,City of Bhangra Festival 2015,0,Festival,NaN,2015-06-01,2015-06-06,NaN,NaN,Vancouver
184,184,Music Waste Festival,0,Festival,NaN,2015-06-04,2015-06-07,NaN,NaN,Vancouver
185,185,Bard on the Beach Shakespeare Festival,0,Festival,NaN,2015-06-04,2015-09-26,NaN,NaN,Vancouver
186,186,Pender Harbour Blues Festival,0,Festival,NaN,2015-06-05,2015-06-07,NaN,NaN,Vancouver
187,187,Surrey Greek Food Festival,0,Festival,NaN,2015-06-05,2015-06-14,NaN,NaN,Vancouver
194,194,Ladner Bandfest,1,Festival,NaN,2015-06-06,2015-06-07,NaN,NaN,Vancouver
195,195,Vancouver Mini Maker Faire,1,Festival,NaN,2015-06-06,2015-06-07,NaN,NaN,Vancouver
204,204,BC Renaissance Festival,0,Festival,NaN,2015-06-11,2015-06-14,NaN,NaN,Vancouver


In [56]:
output = []
for row in events.itertuples():
    for date in pd.date_range(row.startDate, row.endDate):
        output.append({
            'eventKey': row.eventKey,
            'name': row.name,
            'isOffWork': row.isOffWork,
            'type': row.type,
            'startDate': row.startDate,
            'endDate': row.endDate,
            'startTime': row.startTime,
            'endTime': row.endTime,
            'city': row.city,
            'date': date
        })
df = pd.DataFrame(output)
display(df)

,city,date,endDate,endTime,eventKey,isOffWork,name,startDate,startTime,type
0,Vancouver,2014-10-08,2014-10-08,NaN,0,0,Hockey,2014-10-08,22:00:00,Sport
1,Vancouver,2014-10-11,2014-10-11,NaN,1,1,Hockey,2014-10-11,22:00:00,Sport
2,Vancouver,2014-10-17,2014-10-17,NaN,2,0,Hockey,2014-10-17,21:30:00,Sport
3,Vancouver,2014-10-18,2014-10-18,NaN,3,1,Hockey,2014-10-18,22:00:00,Sport
4,Vancouver,2014-10-21,2014-10-21,NaN,4,0,Hockey,2014-10-21,20:30:00,Sport
5,Vancouver,2014-10-23,2014-10-23,NaN,5,0,Hockey,2014-10-23,20:00:00,Sport
6,Vancouver,2014-10-24,2014-10-24,NaN,6,0,Hockey,2014-10-24,21:00:00,Sport
7,Vancouver,2014-10-26,2014-10-26,NaN,7,1,Hockey,2014-10-26,21:30:00,Sport
8,Vancouver,2014-10-26,2014-10-26,NaN,8,1,Hockey,2014-10-26,22:00:00,Sport
9,Vancouver,2014-10-28,2014-10-28,NaN,9,0,Hockey,2014-10-28,22:00:00,Sport


In [57]:
display(df.loc[df['startDate'] != df['endDate']])

,city,date,endDate,endTime,eventKey,isOffWork,name,startDate,startTime,type
161,Denver,2015-05-22,2015-05-24,NaN,161,0,MeadowGrass Music Festival,2015-05-22,NaN,Festival
162,Denver,2015-05-23,2015-05-24,NaN,161,0,MeadowGrass Music Festival,2015-05-22,NaN,Festival
163,Denver,2015-05-24,2015-05-24,NaN,161,0,MeadowGrass Music Festival,2015-05-22,NaN,Festival
179,Vancouver,2015-06-01,2015-06-30,NaN,177,0,Italian Heritage Month,2015-06-01,NaN,Festival
180,Vancouver,2015-06-02,2015-06-30,NaN,177,0,Italian Heritage Month,2015-06-01,NaN,Festival
181,Vancouver,2015-06-03,2015-06-30,NaN,177,0,Italian Heritage Month,2015-06-01,NaN,Festival
182,Vancouver,2015-06-04,2015-06-30,NaN,177,0,Italian Heritage Month,2015-06-01,NaN,Festival
183,Vancouver,2015-06-05,2015-06-30,NaN,177,0,Italian Heritage Month,2015-06-01,NaN,Festival
184,Vancouver,2015-06-06,2015-06-30,NaN,177,0,Italian Heritage Month,2015-06-01,NaN,Festival
185,Vancouver,2015-06-07,2015-06-30,NaN,177,0,Italian Heritage Month,2015-06-01,NaN,Festival


In [58]:
df.to_csv('datasets/events_mapping.csv', index=False)

In [59]:
df.drop(['city'], axis=1, inplace=True)
df.to_csv('datasets/final/events.csv', index=False)